# Merging Survey and Platform Data -  Group with Randomization

Completely done in Python. There ore no steps required in Excel

v2_18.04.2024

In [1]:
import pandas as pd

In [16]:
# Load the first dataset
df1 = pd.read_csv('data_prep_constructs_survey.csv')

# Load the second dataset
df2 = pd.read_csv('data_prep_cleaned_platform.csv')

In [17]:
# Merge the datasets on 'VPNCode'
merged_df = pd.merge(df1, df2[['VPNCode', 'group_nr', 'All_Prompts', 'Prompt_Count', 'Average_Word_Count', 'First_Prompt_Word_Count', 'Total_Word_Count']], on='VPNCode', how='inner')

In [18]:
# Treat 'group_nr' as a categorical variable
merged_df['group_nr'] = merged_df['group_nr'].astype('category')

In [19]:
merged_df.head()

,VPNCode,AGE,WORK,Gender_Männlich,Gender_Weiblich,Edu_Abgeschlossene_Berufsausbildung,Edu_Mittlere_Reife_(Realschulabschluss),Edu_Abitur_oder_Fachabitur,Edu_Bachelor,Edu_Master,...,time_task1,time_survey1,time_task2,time_survey2,group_nr,All_Prompts,Prompt_Count,Average_Word_Count,First_Prompt_Word_Count,Total_Word_Count
0,ZZEBA0308,25.0,100.0,False,True,False,False,False,True,False,...,6.123167,4.870667,0.053667,3.967667,6.0,Bitte hilf mir bei folgender aufgabe:\nEs ist ...,1.0,116.0,116.0,116.0
1,ZZNRO406,30.0,100.0,False,True,False,False,False,True,False,...,9.116833,4.535500,6.950500,3.373833,4.0,"Moin, du sollst mir bitte freundlich aber seri...",3.0,15.0,21.0,45.0
2,ZZSSA0301,22.0,0.0,False,True,False,False,True,False,False,...,3.944333,4.997500,2.787167,6.448333,5.0,0,0.0,0.0,0.0,0.0
3,ZZAWA0212,22.0,100.0,True,False,False,False,True,False,False,...,5.711333,3.551500,0.067667,6.050667,3.0,Bitte lösen Sie die folgende Aufgabe\nEs ist F...,1.0,132.0,132.0,132.0
4,ZZKO1608,37.0,100.0,True,False,False,False,False,True,False,...,5.228333,3.087500,0.074333,6.143000,2.0,Entwickeln Sie ein Konzept für eine denkwürdig...,1.0,54.0,54.0,54.0


In [20]:
# Adress Radomization by exchaning values from the following columns if the 'group_nr' values are 5, 6, 7, and 8 (from 1 to 4 the order is already correct)
# Define the columns to swap in pairs
swap_pairs = [
    ('JC1[HRJD]', 'JC2[HRJD]'),
    ('JC1[IStR]', 'JC2[IStR]'),
    ('MANI1', 'MANI2'),
    ('PROD1', 'PROD2'),
    ('SDT1[AUT]', 'SDT2[AUT]'),
    ('SDT1[COM]', 'SDT2[COM]'),
    ('SE1', 'SE2'),
    ('TASKDIF1', 'TASKDIF2'),
    ('time_survey1', 'time_survey2'),
    ('time_task1', 'time_task2')
]

# Loop through each pair and swap their values for rows with group_nr in 5, 6, 7, 8
for col1, col2 in swap_pairs:
    # Identify rows where swapping is needed
    mask = merged_df['group_nr'].isin([5, 6, 7, 8])
    
    # Perform the swap
    merged_df.loc[mask, [col1, col2]] = merged_df.loc[mask, [col2, col1]].values

In [21]:
# Create a binary 'Prompt_Support' column with 0 as default value
merged_df['Prompt_Support'] = 0

# Update 'Prompt_Support' to 1 where 'group_nr' is 2, 4, 6, 8 (= Prompting Framework used)
merged_df.loc[merged_df['group_nr'].isin([2, 4, 6, 8]), 'Prompt_Support'] = 1

In [22]:
# Rename some columns for convinience
merged_df.rename(columns={
    'Average_Word_Count': 'Average_Word_Count_Prompts',
    'Total_Word_Count': 'Total_Word_Count_Prompt',
    'VPNCode': 'ID',
    'group_nr': 'Group_Nr',
    'Edu_Mittlere_Reife_(Realschulabschluss)' : 'Edu_Mittlere_Reife_Realschulabschluss'
}, inplace=True)

In [23]:
# Print the list of column names
column_names = merged_df.columns.tolist()
print(column_names)

['ID', 'AGE', 'WORK', 'Gender_Männlich', 'Gender_Weiblich', 'Edu_Abgeschlossene_Berufsausbildung', 'Edu_Mittlere_Reife_Realschulabschluss', 'Edu_Abitur_oder_Fachabitur', 'Edu_Bachelor', 'Edu_Master', 'Edu_Doktortitel', 'AILiteracy[Use]', 'AILiteracy[Kno]', 'AILiteracy[Det]', 'AILiteracy[Eth]', 'PGAT', 'NGAT', 'CMV', 'NEO[E]', 'NEO[A]', 'NEO[C]', 'NEO[N]', 'NEO[O]', 'JC1[IStR]', 'JC1[HRJD]', 'SE1', 'SDT1[AUT]', 'SDT1[COM]', 'PROD1', 'TASKDIF1', 'MANI1', 'JC2[IStR]', 'JC2[HRJD]', 'SE2', 'SDT2[AUT]', 'SDT2[COM]', 'PROD2', 'TASKDIF2', 'MANI2', 'total_time', 'syst_total_time', 'time_demogr', 'time_pers', 'time_task1', 'time_survey1', 'time_task2', 'time_survey2', 'Group_Nr', 'All_Prompts', 'Prompt_Count', 'Average_Word_Count_Prompts', 'First_Prompt_Word_Count', 'Total_Word_Count_Prompt', 'Prompt_Support']


In [25]:
# Change order of columns to a more desired one in order to get a better overview
new_column_order = [
    'ID', 'AGE', 'WORK', 'Gender_Männlich', 'Gender_Weiblich', 
    'Edu_Abgeschlossene_Berufsausbildung', 'Edu_Mittlere_Reife_Realschulabschluss', 'Edu_Abitur_oder_Fachabitur', 'Edu_Bachelor', 'Edu_Master', 'Edu_Doktortitel',
    'AILiteracy[Use]', 'AILiteracy[Kno]', 'AILiteracy[Det]', 'AILiteracy[Eth]', 'PGAT', 'NGAT', 'CMV', 
    'NEO[E]', 'NEO[A]', 'NEO[C]', 'NEO[N]', 'NEO[O]', 'Group_Nr', 'Prompt_Support',
    'JC1[IStR]', 'JC1[HRJD]', 'SE1', 'SDT1[AUT]', 'SDT1[COM]', 'PROD1', 'TASKDIF1', 'MANI1',
    'JC2[IStR]', 'JC2[HRJD]', 'SE2', 'SDT2[AUT]', 'SDT2[COM]', 'PROD2', 'TASKDIF2', 'MANI2', 
    'All_Prompts', 'Prompt_Count', 'Average_Word_Count_Prompts', 'First_Prompt_Word_Count', 'Total_Word_Count_Prompt',
    'total_time', 'syst_total_time', 'time_demogr', 'time_pers', 'time_task1', 'time_survey1', 'time_task2', 'time_survey2'
]

merged_df = merged_df[new_column_order]

In [26]:
# Maybe not the big focus for my main analysis but could be useful for the explorative analysis: Time columns don't seem always correct, so some changes will be implemented

# Drop the 'total_time' column as it exists twice (= syst_total_time)
merged_df.drop(columns=['total_time'], inplace=True)

# Rename 'syst_total_time' column to 'time_total' for coherency
merged_df.rename(columns={'syst_total_time': 'time_total'}, inplace=True)

# Compare 'time_task1' with 'time_survey1' and swap if 'time_task1' is smaller 
# Reason: I could be wrong but looking at the values from the corrresponding columns, it does seem more resonable that someone required more time to complete the task than the survey (further, sometimes absurd values like 4 seconds for the task but 11 minutes for the survey). This is not the case for every row
mask = merged_df['time_task1'] < merged_df['time_survey1']
merged_df.loc[mask, ['time_task1', 'time_survey1']] = merged_df.loc[mask, ['time_survey1', 'time_task1']].values

# Compare 'time_task2' with 'time_survey2' and swap if 'time_task2' is smaller 
# Same reason as above
mask = merged_df['time_task2'] < merged_df['time_survey2']
merged_df.loc[mask, ['time_task2', 'time_survey2']] = merged_df.loc[mask, ['time_survey2', 'time_task2']].values

C:\Users\franc\AppData\Local\Temp\ipykernel_13580\65296367.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.drop(columns=['total_time'], inplace=True)
C:\Users\franc\AppData\Local\Temp\ipykernel_13580\65296367.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.rename(columns={'syst_total_time': 'time_total'}, inplace=True)


In [27]:
# Renaming columns to remove brackets (especially for R)
merged_df.columns = [col.replace('[', '').replace(']', '') for col in merged_df.columns]

In [28]:
# Look at the new dataset
merged_df.head()

,ID,AGE,WORK,Gender_Männlich,Gender_Weiblich,Edu_Abgeschlossene_Berufsausbildung,Edu_Mittlere_Reife_Realschulabschluss,Edu_Abitur_oder_Fachabitur,Edu_Bachelor,Edu_Master,...,Average_Word_Count_Prompts,First_Prompt_Word_Count,Total_Word_Count_Prompt,time_total,time_demogr,time_pers,time_task1,time_survey1,time_task2,time_survey2
0,ZZEBA0308,25.0,100.0,False,True,False,False,False,True,False,...,116.0,116.0,116.0,21.029833,4.679333,1.335333,3.967667,0.053667,6.123167,4.870667
1,ZZNRO406,30.0,100.0,False,True,False,False,False,True,False,...,15.0,21.0,45.0,29.111167,4.026833,1.107667,9.116833,4.535500,6.950500,3.373833
2,ZZSSA0301,22.0,0.0,False,True,False,False,True,False,False,...,0.0,0.0,0.0,25.566000,5.850667,1.538000,6.448333,2.787167,4.997500,3.944333
3,ZZAWA0212,22.0,100.0,True,False,False,False,True,False,False,...,132.0,132.0,132.0,20.772667,3.890000,1.501500,5.711333,3.551500,6.050667,0.067667
4,ZZKO1608,37.0,100.0,True,False,False,False,False,True,False,...,54.0,54.0,54.0,19.773667,4.368833,0.871667,5.228333,3.087500,6.143000,0.074333


In [29]:
# Save DF in a new CSV-File 
merged_df.to_csv('data_prep_analysis_prolific.csv', index=False, encoding='utf-8-sig', sep=',')